<a href="https://colab.research.google.com/github/Roastedalmonds/CourseraML_Project/blob/master/MaC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version  2.x  
!pip install -q sklearn #sklearn package 
import tensorflow as tf

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output   #clears output of current cell recieving the output
from six.moves import urllib    #url handling modules

import tensorflow.compat.v2.feature_column as fc

In [0]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

In [0]:
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')


In [0]:
CATEGORIAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']
feature_columns=[]

for feature_name in CATEGORIAL_COLUMNS:   #stores unique values of features within feature_columns
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

#print(feature_columns)

In [6]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle = True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df)) #create data.Dataset object
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)#creating trainer dataset object by passing og feature and label
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs = 1, shuffle= False)#creating tester dataset object

linear_est = tf.estimator.LinearClassifier(feature_columns= feature_columns) 

linear_est.train(train_input_fn)#training data object model

result = linear_est.evaluate(eval_input_fn) #dictionary with various o/p characteristics
clear_output()
print(result['accuracy'])


0.7537879


In [20]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[136])
print(y_eval.loc[136])
print(result[136]['probabilities'][1])

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbrx9b7q2/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                          male
age                            55
n_siblings_spouses              0
parch                           0
fare                         30.5
class                       First
deck                            C
emb